### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [50]:
reviews.sample(5)

,user_id,movie_id,rating,timestamp,date
770993,60743,1518812,7,1572878661,2019-11-04 16:44:21
761391,59986,963178,7,1371986924,2013-06-23 13:28:44
809683,63818,2510894,9,1452832146,2016-01-15 06:29:06
473625,37336,7664504,9,1546479127,2019-01-03 03:32:07
769987,60712,2084970,10,1422090674,2015-01-24 11:11:14


In [51]:
# Function to get average rating
def get_reviews_avg_nr(ival):
    temp_rev = reviews[reviews['movie_id'] == ival]
    i_nr_of_ratings = len(temp_rev)

    if i_nr_of_ratings > 0:
        i_avg_rating = np.average(temp_rev['rating'])
        i_last_rating = temp_rev['date'].max()
    else: 
        i_avg_rating = np.nan
        i_last_rating = np.nan
    
    return i_avg_rating, i_nr_of_ratings, i_last_rating

In [52]:
def get_top_movie_ratings():
    movies['average_rating'] = np.nan
    movies['nr_of_reviews'] = np.nan
    movies['last_review'] = np.nan

    for i in range(len(movies)):
        iavg_rating, i_nr_of_ratings, ilast_rating = get_reviews_avg_nr(movies['movie_id'][i])
        movies.iloc[i, movies.columns.get_loc('average_rating')] = iavg_rating
        movies.iloc[i, movies.columns.get_loc('nr_of_reviews')] = i_nr_of_ratings
        movies.iloc[i, movies.columns.get_loc('last_review')] = ilast_rating    

    movies_rankings = movies[movies['nr_of_reviews'] >= 4]
    
    return(movies_rankings)

global_movies_rankings = get_top_movie_ratings()

In [53]:
def popular_recommendations(user_id, n_top, movies_rankings):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    movies_rankings = movies_rankings.sort_values(by=['average_rating', 'nr_of_reviews', 'last_review'], ascending=False)
    top_movies = movies_rankings[0:n_top]
    
    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [54]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1
recs_20_for_1 = popular_recommendations(1, 20, global_movies_rankings)

# # Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5, global_movies_rankings)

# # Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000, 100, global_movies_rankings)

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43, 35, global_movies_rankings)



In [55]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

AttributeError: module 'tests' has no attribute 'create_ranked_df'

**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [ ]:
movie_rankings = global_movies_rankings

years = ['2015', '2016', '2017', '2018']

if len(years) > 0:
    movie_rankings = movie_rankings[movie_rankings['date'].isin(years)]    
    
movie_rankings.head()

In [ ]:
genres = ['Animation', 'Short']

if len(genres) > 0:
    movies_rankings_genres = []
for i in range(len(genres)):
    ifiltered = movie_rankings[movie_rankings[genres[i]] == 1]
    if len(movies_rankings_genres) == 0:
        movies_rankings_genres = ifiltered
    else:    
        movies_rankings_genres = pd.concat([movies_rankings_genres, ifiltered])
    




In [ ]:
def popular_recs_filtered(user_id, n_top, movies_rankings, years, genres):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''

    movies_rankings = movies_rankings.sort_values(by=['average_rating', 'nr_of_reviews', 'last_review'], ascending=False)

    # filter years                         
    if len(years) > 0:
        movies_rankings = movies_rankings[movies_rankings['date'].isin(years)]                              

    # filter by genres
    if len(genres) > 0:
        movies_rankings_genres = []
        for i in range(len(genres)):
            ifiltered = movies_rankings[movies_rankings[genres[i]] == 1]
            if len(movies_rankings_genres) == 0:
                movies_rankings_genres = ifiltered
            else:    
                movies_rankings_genres = pd.concat([movies_rankings_genres, ifiltered])
        movies_rankings = movies_rankings_genres
        
    top_movies = movies_rankings[0:n_top]
    
    return top_movies # a list of the n_top movies as recommended

In [ ]:
popular_recs_filtered('1', 5, global_movies_rankings, years=['2015', '2016', '2017', '2018'], genres=['Animation'])

In [ ]:
num_genre_match = global_ranked_movies[genres].sum(axis=1)
ranked_movies = global_ranked_movies.loc[num_genre_match > 0, :]